In [ ]:
# Here is the imports
import os
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'
FLAGS = [] # tensorboard, mixed_precision
from tensorflow import keras
import numpy as np
import tensorflow as tf
if "mixed_precision" in FLAGS:
    print(f"PreTrain: Using Mixed Policy float16")
    keras.mixed_precision.set_global_policy('mixed_float16')
from matplotlib import pyplot as plt
AUTOTUNE = tf.data.AUTOTUNE
import random
import albumentations as A
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from tensorflow.keras.applications import EfficientNetB4
import efficientnet.tfkeras as eff

In [ ]:
DATASET_PATH = "./recordbase"
TRAIN_DIR = "train"
VAL_DIR = "val"
TEST_DIR = "test"

RECORD_ENCODING_TYPE = "ZLIB" # none if no encoding is used

# Pipeline parameters
BUFFER_SIZE = None # set buffer size to default value, change if you have bottleneck
SHUFFLE_SIZE = 256 # because dataset is too large huge shuffle sizes may cause problems with ram
BATCH_SIZE = 2 # Highly dependent on d-gpu and system ram
STEPS_PER_EPOCH = 4977//BATCH_SIZE # 4646 IMPORTANT this value should be equal to file_amount/batch_size because we can't find file_amount from tf.Dataset you should note it yourself
VAL_STEPS_PER_EPOCH = 1659//BATCH_SIZE # 995 same as steps per epoch
MODEL_WEIGHTS_PATH = None#'./models/14_09-22_55/best.h5' # if not none model will be contiune training with these weights
# every shard is 200 files with 36 files on last shard
# Model Constants
BACKBONE = 'efficientnetb3'
# inme_yok, inme_var
CLASSES = ['inme_yok', 'inme_var']
LR = 0.0001
IMG_SIZE = 512
FIRST_EPOCHS = 100
FINE_TUNE_EPOCHS = 100
MODEL_SAVE_PATH = "./models"

In [ ]:
specifier_name = 'phase1_efficent_net'
date_name = f'{datetime.now().strftime("%d_%m-%H_%M")}-{specifier_name}'

# Variables
train_dir = os.path.join(DATASET_PATH, TRAIN_DIR)
val_dir = os.path.join(DATASET_PATH, VAL_DIR)

train_filenames = tf.io.gfile.glob(f"{train_dir}/*.tfrecords")
val_filenames = tf.io.gfile.glob(f"{val_dir}/*.tfrecords")

random.shuffle(train_filenames) # shuffle tfrecord files order
random.shuffle(val_filenames)

os.makedirs(f'{MODEL_SAVE_PATH}/{date_name}', exist_ok=True)

# define callbacks for learning rate scheduling and best checkpoints saving
callbacks = [
    keras.callbacks.ModelCheckpoint(f'{MODEL_SAVE_PATH}/{date_name}/best.h5', save_weights_only=False, save_best_only=True, mode='min'),
    keras.callbacks.ModelCheckpoint(f'{MODEL_SAVE_PATH}/{date_name}/epoch_{{epoch:02d}}.h5', save_weights_only=False, save_freq=STEPS_PER_EPOCH*10, save_best_only=False, mode='min'),
    #keras.callbacks.ModelCheckpoint(f'{MODEL_SAVE_PATH}/{date_name}/weights_{{epoch:02d}}.h5', save_weights_only=True, save_freq=STEPS_PER_EPOCH*5, save_best_only=False, mode='min'),
    keras.callbacks.ReduceLROnPlateau(),
    keras.callbacks.CSVLogger(f'./customlogs/{date_name}.csv')
]

if "tensorboard" in FLAGS:
    print(f"PreTrain: Using tensorboard")
    callbacks.append(
        TensorBoard(
            log_dir="logs",
            histogram_freq=0,
            write_graph=True,
            write_images=False,
            update_freq="epoch",
        ))

In [ ]:
def aug_fn(image):
    transforms = A.Compose([
            A.Rotate(limit=40),
            A.Flip(),
            ])
    aug_data = transforms(image=image)
    aug_img, aug_mask = aug_data["image"]
    return aug_img, aug_mask

def preprocessing_fn(image, mask):
    image, mask = image.astype("float32"), mask.astype("float32")
    return image, mask 

def parse_examples_batch(examples):
    feature_description = {
        'image' : tf.io.FixedLenFeature([], tf.string),
        'label' : tf.io.FixedLenFeature([], tf.int64)
    }
    samples = tf.io.parse_example(examples, feature_description)
    return samples

def prepare_sample(features):
    image = tf.vectorized_map(lambda x: tf.io.parse_tensor(x, out_type = tf.uint8), features["image"])
    label = tf.vectorized_map(lambda x: tf.io.parse_tensor(x, out_type = tf.int64), features["label"])
    #image, label = tf.vectorized_map(lambda x: tf.numpy_function(func=preprocessing_fn, inp=x, Tout=(tf.float32, tf.int64)), [image, label])
    return image, label

def prepare_sample_aug(features):
    image = tf.vectorized_map(lambda x: tf.io.parse_tensor(x, out_type = tf.uint8), features["image"])
    label = tf.vectorized_map(lambda x: tf.io.parse_tensor(x, out_type = tf.int64), features["label"]) # this was float64
    image = tf.vectorized_map(lambda x: tf.numpy_function(func=aug_fn, inp=x, Tout=(tf.uint8)), image)
    #image, label = tf.vectorized_map(lambda x: tf.numpy_function(func=preprocessing_fn, inp=x, Tout=(tf.float32, tf.float32)), [image, label])
    return image, label

def get_dataset_optimized(filenames, batch_size, epoch_num, shuffle_size, augment=True):
    record_dataset = tf.data.TFRecordDataset(filenames, compression_type=RECORD_ENCODING_TYPE, num_parallel_reads=AUTOTUNE)
    if shuffle_size > 0:
        record_dataset = record_dataset.shuffle(shuffle_size)
    record_dataset = (record_dataset
                    .repeat(epoch_num)
                    .batch(batch_size=batch_size)
                    .map(map_func=parse_examples_batch, num_parallel_calls=tf.data.experimental.AUTOTUNE))
    if augment:
        record_dataset = record_dataset.map(map_func=prepare_sample_aug, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    else:
        record_dataset = record_dataset.map(map_func=prepare_sample, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return record_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

model = EfficientNetB4(
    include_top=False,
    weights="imagenet",
    input_tensor=inputs,
    input_shape=(512, 512, 3),
)

#model = eff.EfficientNetB4(include_top=False,
#        weights='noisy-student',
#        input_tensor=x,
#        input_shape=(512, 512, 3))

model.trainable = False

# Rebuild top
x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
x = layers.BatchNormalization()(x)

top_dropout_rate = 0.2
x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = layers.Dense(1, activation="sigmoid", name="pred")(x)

optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
model.compile(
    optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=["accuracy"]
)

In [ ]:
history = model.fit(
        get_dataset_optimized(train_filenames, BATCH_SIZE, FIRST_EPOCHS, SHUFFLE_SIZE, augment=True), 
        steps_per_epoch=STEPS_PER_EPOCH, 
        epochs=FIRST_EPOCHS, 
        callbacks=callbacks, 
        validation_data=get_dataset_optimized(val_filenames, BATCH_SIZE, FIRST_EPOCHS, 0, augment=False), 
        validation_steps=VAL_STEPS_PER_EPOCH,
        #initial_epoch=5
    )

In [ ]:
for layer in model.layers[-20:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True
    optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
    model.compile(
    optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=["accuracy"]
)

In [ ]:
history = model.fit(
        get_dataset_optimized(train_filenames, BATCH_SIZE, FIRST_EPOCHS, SHUFFLE_SIZE, augment=True), 
        steps_per_epoch=STEPS_PER_EPOCH, 
        epochs=FINE_TUNE_EPOCHS, 
        callbacks=callbacks, 
        validation_data=get_dataset_optimized(val_filenames, BATCH_SIZE, FIRST_EPOCHS, 0, augment=False), 
        validation_steps=VAL_STEPS_PER_EPOCH,
        initial_epoch=FIRST_EPOCHS
    )